# 06 - Combining data with merges

In [253]:
import pandas as pd
import math

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## 1. Match candidate districts from the FEC with demographic data from the census

### a. Load candidate and demographic files to identify common attributes

In [257]:
candidate_header = pd.read_csv('downloaded_data/cn_header_file.csv').columns.tolist()
candidates = pd.read_csv('downloaded_data/cn22.txt', sep='|', names=candidate_header)

In [259]:
candidates.head(2)

,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP
0,H0AK00105,"LAMB, THOMAS",NNE,2020,AK,H,0.0,C,N,C00607515,1861 W LAKE LUCILLE DR,NaN,WASILLA,AK,99654.0
1,H0AL01055,"CARL, JERRY LEE, JR",REP,2022,AL,H,1.0,I,C,C00697789,PO BOX 852138,NaN,MOBILE,AL,36685.0


In [230]:
districts = pd.read_csv('downloaded_data/ACSDT5Y2019.B01003_2021-07-14T121439/ACSDT5Y2019.B01003_data_with_overlays_2021-07-14T121436.csv', header=1)

In order to join candidates and census data, we need the identifying columns to be in the same format and variable type. We can do a double merge or we can create a new key with district and state abbreviation in each table

### b. Format candidates table

In [229]:
candidates['district_str'] = candidates['CAND_OFFICE_DISTRICT'].apply(lambda x: x if math.isnan(x) else str(round(x)).zfill(2))

### c. Format demographics table

How can we identify districts and states the way that the FEC identifies? Split the id column into state ID and district ID

[Read up on splicing notation here](https://www.oreilly.com/content/how-do-i-use-the-slice-notation-in-python/)

[Find FIPs codes here](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696)

In [231]:
districts['district_id'] = districts['id'].str[-2:]

In [232]:
districts['state_fips'] = districts['id'].str[-4:-2]

Now we need the state abbreviation instead of the fips code. We need to look up that value

In [233]:
state_lookup = pd.read_html('https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696')
state_lookup = state_lookup[0]
state_lookup = state_lookup.drop(index=[55])

# FIPS codes need to be have a length of 2 and exist in strings
state_lookup['FIPS'] = state_lookup['FIPS'].apply(lambda x: str(round(x)).zfill(2))

In [234]:
state_lookup

,Name,Postal Code,FIPS
0,Alabama,AL,01
1,Alaska,AK,02
2,Arizona,AZ,04
3,Arkansas,AR,05
4,California,CA,06
5,Colorado,CO,08
6,Connecticut,CT,09
7,Delaware,DE,10
8,Florida,FL,12
9,Georgia,GA,13


In [235]:
districts = districts.merge(state_lookup, left_on='state_fips', right_on='FIPS', how='left')

In [236]:
districts.head(2)

,id,Geographic Area Name,Estimate!!Total,Margin of Error!!Total,district_id,state_fips,Name,Postal Code,FIPS
0,5001600US0101,"Congressional District 1 (116th Congress), Ala...",710135.0,615,01,01,Alabama,AL,01
1,5001600US0102,"Congressional District 2 (116th Congress), Ala...",679684.0,2213,02,01,Alabama,AL,01


Create a common key between the two tables now that columns are formatted

In [237]:
districts['key'] = districts['district_id'] + districts['Postal Code']

In [238]:
candidates['key'] = candidates['district_str'] + candidates['CAND_OFFICE_ST']

### d. Merge formatted tables

We need the identifying columns to be in the same format and variable type. We can do a double merge or we can create a new key with district and state abbreviation in each table.

[Read about merging in pandas here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)

Identify...

- Left table:
- Right table:
- Left key:
- Right key:
- Merge type:

In [239]:
candidates_districts = candidates.merge(districts, on='key', how='left')

In [240]:
len(candidates_districts), len(districts), len(candidates)

(5377, 440, 5377)

In [241]:
candidates_districts

,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP,district_str,key,id,Geographic Area Name,Estimate!!Total,Margin of Error!!Total,district_id,state_fips,Name,Postal Code,FIPS
0,H0AK00105,"LAMB, THOMAS",NNE,2020,AK,H,0.0,C,N,C00607515,1861 W LAKE LUCILLE DR,NaN,WASILLA,AK,99654.0,00,00AK,5001600US0200,Congressional District (at Large) (116th Congr...,737068.0,*****,00,02,Alaska,AK,02
1,H0AL01055,"CARL, JERRY LEE, JR",REP,2022,AL,H,1.0,I,C,C00697789,PO BOX 852138,NaN,MOBILE,AL,36685.0,01,01AL,5001600US0101,"Congressional District 1 (116th Congress), Ala...",710135.0,615,01,01,Alabama,AL,01
2,H0AL01063,"LAMBERT, DOUGLAS WESTLEY III",REP,2020,AL,H,1.0,O,P,C00701557,7194 STILLWATER BLVD,NaN,SPANISH FORT,AL,36527.0,01,01AL,5001600US0101,"Congressional District 1 (116th Congress), Ala...",710135.0,615,01,01,Alabama,AL,01
3,H0AL01097,"AVERHART, JAMES",DEM,2020,AL,H,1.0,O,P,C00708867,430 SAINT LOUIS STREET,NaN,MOBILE,AL,36602.0,01,01AL,5001600US0101,"Congressional District 1 (116th Congress), Ala...",710135.0,615,01,01,Alabama,AL,01
4,H0AL02087,"ROBY, MARTHA",REP,2020,AL,H,2.0,I,P,C00462143,NaN,NaN,MONTGOMERY,NaN,NaN,02,02AL,5001600US0102,"Congressional District 2 (116th Congress), Ala...",679684.0,2213,02,01,Alabama,AL,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5372,S8WI00224,"VUKMIR, LEAH",REP,2018,WI,S,0.0,C,P,C00655522,945 POST RD,NaN,BROOKFIELD,WI,53005.0,00,00WI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5373,S8WV00119,"SWEARENGIN, PAULA JEAN",DEM,2018,WV,S,0.0,C,P,C00708891,127 SAPPHIRE LANE,NaN,COAL CITY,WV,25823.0,00,00WV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5374,S8WV00127,"JENKINS, EVAN H",REP,2018,WV,S,0.0,C,P,C00548271,121 OAK LANE,NaN,HUNTINGTON,WV,25701.0,00,00WV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5375,S8WV00135,"COPLEY, JIMMY DALE II",REP,2018,WV,S,0.0,C,N,C00650564,92 BOY SCOUT HOLLOW,NaN,DELBARTON,WV,25670.0,00,00WV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Clean data
We would want Senate candidates to be matched with the state as a whole or else there will be no identifier in the Census table or they would be erroneously matched to states with "at large" districts.

In [260]:
# filter by house candidates only
candidates_house = candidates[candidates['CAND_OFFICE'] == 'H']

In [262]:
# merge again
house_candidates_districts = candidates_house.merge(districts, on='key', how='left')

In [244]:
len(house_candidates_districts)

3578

How can we check everything is matched as we would expect? There should be 440 different districts. How can we find invalid districts?

In [245]:
district_counts = pd.DataFrame(house_candidates_districts.groupby('key')['key'].count())
district_counts.columns = ['district_counts']
district_counts = district_counts.reset_index()

In [252]:
district_counts.sort_values(by='district_counts').head(20)

,key,district_counts
467,90IN,1
232,06OR,1
388,19PA,1
453,43MI,1
181,04WV,1
43,01MP,1
216,06CT,1
426,28FL,1
466,80MI,1
14,00PA,1
